# CH.3 Effective Functions



# 3.1 Python function ares are fist class 
function 是一種 first-class (一級物件)，它可被指派給變數、用資料結構儲存、或是當作 argument 傳遞、甚至當作回傳值。

In [3]:
# This is just a toy function
# add !!!!! after the input string
def yell(text):
  return text + '!!!!!'

x = yell

## Functions Are Objects

In [4]:
bark = yell
bark('woof')

'woof!!!!!'

如果刪除原本的 yell，yell 不能再使用
```python
>>> del yell 

>>> yell('hi')
NameError: "name 'yell' is not defined"
```
但 bark 仍舊指向 underlying function，還是能使用

In [5]:
del yell
bark('hi')

'hi!!!!!'

可透過 `__name__` 來取得 python 給每個 function 的 string identifier
+ function 本體跟指向他的變數 (yell 和 bark) 完全是兩回事

In [6]:
bark.__name__

'yell'

## Functions Can Be Stored in Data Structures

In [7]:
funcs = [bark, str.lower, str.capitalize]
funcs

[<function __main__.yell>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

## Functions Can Be Passed to Other Functions
可以接收其他 function 作為輸入的 function 叫做 `higher-order functions`

In [8]:
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

In [9]:
greet(bark)

Hi, I am a Python program!!!!!


最經典的例子，是 python 內建的 map
+ It takes a function object and an iterable, and then
calls the function on each element in the iterable, yielding the results
as it goes along

In [10]:
list(map(bark, ['hello', 'hey', 'hi']))

['hello!!!!!', 'hey!!!!!', 'hi!!!!!']

## Functions Can Be Nested (巢狀)
以下的範例，在 speak 底下定義了 whisper function，並立即使用

In [11]:
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)

speak('hello world')

'hello world...'

但如果的很想要裡面的子 function，也可以把它給回傳出來 (因為 function 是一種 object)

In [12]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper

## Functions Can Capture Local State
此時的 whisper() 和 yell() 沒有傳入引數了，但還是能抓到 text, volumn 變數  
這是因為 lexical closures (closures) 特性，這兩個 function 會記住區域內的變數

In [13]:
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper

In [14]:
get_speak_func('Hello, World', 0.7)()

'HELLO, WORLD!'

## Objects Can Behave Like Functions
如果想讓 oject 像是 function 一樣可以呼叫，請實作 `__call__()`

In [15]:
class Adder:
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        return self.n + x

plus_3 = Adder(3)
plus_3(4)

7

**Key Takeaways**
+ **Everything in Python is an object, including functions.** You can
assign them to variables, store them in data structures, and pass
or return them to and from other functions (first-class functions.)
+ First-class functions allow you to abstract away and pass
around behavior in your programs.
+ Functions can be nested and they can capture and carry some
of the parent function’s state with them. Functions that do this
are called closures.
+ Objects can be made callable. In many cases this allows you to
treat them like functions.


# 3.2 Lambdas Are Single-Expression Functions

lambda 語法在 python 中用於建立 small anonymous functions ，與 def 不同點在於
+ 不用綁定 function name 就能使用
+ 不能 return ，且只有一個 statement

In [16]:
add = lambda x, y: x + y
add(5, 3)

8

還能這樣使用，定義了 lambda function 後傳入 (5, 3)

In [17]:
(lambda x, y: x + y)(5, 3)

8

常見的 lambda 用法，用於回傳比較基準

In [18]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

使用 lambda 要格外小心，謹慎思考後是否好維護  
  
以下是會讓夥伴扁你的程式碼
```python
# Harmful:
>>> list(filter(lambda x: x % 2 == 0, range(16)))
[0, 2, 4, 6, 8, 10, 12, 14]
# Better:
>>> [x for x in range(16) if x % 2 == 0]
[0, 2, 4, 6, 8, 10, 12, 14]
```

```python
# Harmful:
class Car:
    rev = lambda self: print('Wroom!')
    crash = lambda self: print('Boom!')

>> my_car = Car()
>> my_car.crash()
'Boom!'
```

**Key Takeaways**
+ Lambda functions are single-expression functions that are not
necessarily bound to a name (anonymous).
+ Lambda functions can’t use regular Python statements and always include an implicit return statement.
+ Always ask yourself: Would using a regular (named) function
or a list comprehension offer more clarity?

# 3.3 The Power of Decorators
python decorators 可以在讓你不更改原本 object(functions, methods, and classes) 時延伸或改變其行為

以下情況使用 decorator 會很方便
+ logging
+ enforcing access control and authentication
+ instrumentation and timing functions
+ rate-limiting
+ caching, and more

假如想幫 30 個 function 計時，不可能在每個 function 都加上計算時間的程式碼吧?
+ 只要寫好 decorator，掛在 function 上面就好

## Python Decorator Basics

decorator 的概念如下，如同在 funcion 外再包一層 function
```python
def null_decorator(func):
    return func

def greet():
    return 'Hello!'

greet = null_decorator(greet)
>>> greet()
'Hello!'

# 作為 decorator 的使用方式
@null_decorator
def greet():
    return 'Hello!'

```

## Decorators Can Modify Behavior
uppercase 作為一個 decorator，將輸入 function 原本的結果轉成大寫後回傳

In [33]:
# 回傳原本 function 的 decorator
def null_decorator(func):
    return func

# 轉成大寫的 decorator
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    print(f"wrapper at {wrapper}")
    return wrapper

@uppercase
def greet():
    return 'Hello!'

greet()

wrapper at <function uppercase.<locals>.wrapper at 0x7f1ca2d5a200>


'HELLO!'

**讓我們看看發生啥事情:**  
我們的 uppercase decorator 回傳的並不是原本的 greet function  
而是裡面的 wrapper function

In [34]:
print(greet)
print(null_decorator(greet))

print(uppercase(greet)) # 注意

<function uppercase.<locals>.wrapper at 0x7f1ca2d5a200>
<function uppercase.<locals>.wrapper at 0x7f1ca2d5a200>
wrapper at <function uppercase.<locals>.wrapper at 0x7f1ca2d5a710>
<function uppercase.<locals>.wrapper at 0x7f1ca2d5a710>


## Applying Multiple Decorators to a Function

同時使用多個 decorator，他們的順序是 bottom to top  
就像是你慢慢堆疊 (decorator stacking)，讓你的 function 越來越多功能

In [39]:
def strong(func):
  def wrapper():
    return '<strong>' + func() + '</strong>'
  return wrapper

def emphasis(func):
  def wrapper():
    return '<em>' + func() + '</em>'
  return wrapper

@strong
@emphasis
def greet():
  return 'Hello!'

In [42]:
# decorated_greet = strong(emphasis(greet))
greet()

'<strong><em>Hello!</em></strong>'

## Decorating Functions That Accept Arguments
如果你把之前示範的 decorator ，套在有 argument 的 function 將無法執行
```python
@emphasis
def apple(a, b):
  return str(a+b)

>>> apple(1,2)
>>> TypeError: wrapper() takes 0 positional arguments but 2 were given
```
以下介紹能接收 arrguments 的 decorator，他的運作流程為:  
\* 接收 position argument 後存放在 args 變數內  
\** 接收 keyword argument 後存放在kwargs 變數內  
wrapper closure 接著把這些變數傳遞到 func 上

In [ ]:
def proxy(func):
  def wrapper(*args, **kwargs):
    return func(*args, **kwargs)
  return wrapper

讓我們來看一下延伸應用，我們印出了 `func.__name__` ，並觀察原本 function 的輸出

In [60]:
def trace(func):
  def wrapper(*args, **kwargs):
    print(f'TRACE: calling {func.__name__}() '
          f'with {args}, {kwargs}')
    
    original_result = func(*args, **kwargs)

    print(f'TRACE: {func.__name__}() '
          f'returned {original_result!r}') # !r 在字串左右加上 '
    
    return original_result

  return wrapper

In [61]:
@trace
def say(name, line):
  return f'{name}: {line}'

say('jame', 'bob')

TRACE: calling say() with ('jame', 'bob'), {}
TRACE: say() returned 'jame: bob'


'jame: bob'

## How to Write “Debuggable” Decorators
在一般 function，`__name__` 和 `__doc__`可以看到名稱和說明  
但如果套上了 decorator 就只會看到 wrapper function 的名稱和說明

In [68]:
def greet():
  """Return a friendly greeting."""
  return 'Hello!'

decorated_greet = uppercase(greet)

wrapper at <function uppercase.<locals>.wrapper at 0x7f1c9ac90d40>


In [69]:
print(greet.__name__)
print(greet.__doc__)
print(decorated_greet.__name__)
print(decorated_greet.__doc__)

greet
Return a friendly greeting.
wrapper
None


這在開發時會造成不方便，但我們能用  `functools.wraps` decorator 來解決此問題  
因此建議在任何的 wrapper 都套上 `functools.wraps`

In [76]:
import functools

# 將 wrapper 套上 decorator
def uppercase(func):
  @functools.wraps(func)
  def wrapper():
    return func().upper()
  return wrapper

@uppercase
def greet():
  """Return a friendly greeting."""
  return 'Hello!'

In [77]:
print(greet.__name__)
print(greet.__doc__)

greet
Return a friendly greeting.


**Key Takeaways**
+ Decorators define reusable building blocks you can apply to a
callable to modify its behavior without permanently modifying
the callable itself.
+ The @ syntax is just a shorthand for calling the decorator on
an input function. Multiple decorators on a single function are
applied bottom to top ( decorator stacking ).
+ As a debugging best practice, use the `functools.wraps` helper
in your own decorators to carry over metadata from the undec-
orated callable to the decorated one.
+ Just like any other tool in the software development toolbox,
decorators are not a cure-all(萬靈丹) and they should not be overused.
It’s important to balance the need to “get stuff done” with the
goal of “not getting tangled up in a horrible, unmaintainable
mess of a code base.”

# 3.4 Fun With *args and **kwargs
具有 \*args 和 \**kwargs 的 function 能接收 optional arguments，讓我們寫出靈活的 API  
以下的範例至少需要一個 argument `required` ，但他同時能接收其他的 argument
+ args 收集 position arguments
+ kwargs 收集 keyword arguments

```python
def foo(required, *args, **kwargs):
  print(required)
  if args:
    print(args)
  if kwargs:
    print(kwargs)
    
>>> foo()
TypeError:
"foo() missing 1 required positional arg: 'required'"
>>> foo('hello')
hello
>>> foo('hello', 1, 2, 3)
hello
(1, 2, 3)
>>> foo('hello', 1, 2, 3, key1='value', key2=999)
hello
(1, 2, 3)
{'key1': 'value', 'key2': 999}
```

## Forwarding Optional or Keyword Arguments
\*args 和 \*\*kwargs 可以傳遞或修改內容

```python
def foo(x, *args, **kwargs):
  kwargs['name'] = 'Alice'
  new_args = args + ('extra', )
  bar(x, *new_args, **kwargs)
```

這在建立 sub-class 時很有幫助，可以在不更改 child class constructor 下傳遞參數給 parent class  
  
AlwaysBlueCar 在建立時，傳遞了所有 argument 給 parent class ，並改寫 internal attribute  
即使 parent class 改了 constructor，我們仍然能繼續使用

In [79]:
class Car:
  def __init__(self, color, mileage):
    self.color = color
    self.mileage = mileage
  
class AlwaysBlueCar(Car):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.color = 'blue'

In [80]:
AlwaysBlueCar('green', 48392).color

'blue'

這也能幫助我們寫出更靈活的 decorator 

In [82]:
def trace(f):
  @functools.wraps(f)
  def decorated_function(*args, **kwargs):
    print(f, args, kwargs)
    result = f(*args, **kwargs)
    print(result)
  return decorated_function

@trace
def greet(greeting, name):
  return '{}, {}!'.format(greeting, name)
  
greet('Hello', 'Bob')

<function greet at 0x7f1c9acc2b00> ('Hello', 'Bob') {}
Hello, Bob!


**Key Takeaways**
+ \*args and \*\*kwargs let you write functions with a variable
number of arguments in Python.
+ \*args collects extra positional arguments as a tuple. \*\*kwargs
collects the extra keyword arguments as a dictionary.
+ The actual syntax is \* and \*\* . Calling them args and kwargs is
just a convention (and one you should stick to).

# 3.5 Function Argument Unpacking
可以透過 \* 和 \** 符號來 unpack function argument 

In [84]:
def print_vector(x, y, z):
  print('<%s, %s, %s>' % (x, y, z))

print_vector(0, 1, 0)

<0, 1, 0>


但如果改變了輸入的格式，這樣使用會很不彈性

In [86]:
tuple_vec = (1, 0, 1)
list_vec = [1, 0, 1]

print_vector(tuple_vec[0], tuple_vec[1], tuple_vec[2])

<1, 0, 1>


此時能透過 *Function Argument Unpacking* 來解決  
把 \* 符號放在 iterable 變數前面，就能作為 positional argument 傳遞  
如果要 unpacking 字典類型則要用 \*\* 符號

In [87]:
print_vector(*tuple_vec)

<1, 0, 1>


In [88]:
dict_vec = {'y': 0, 'z': 1, 'x': 1}
print_vector(**dict_vec)

<1, 0, 1>


**Key Takeaways**
+ The * and ** operators can be used to “unpack” function argu-
ments from sequences and dictionaries.
+ Using argument unpacking effectively can help you write more
flexible interfaces for your modules and functions.

# 3.6 Nothing to Return Here
如果 function 沒指定 return 什麼東西，預設就是 None  
作者建議如果沒 return value，那就把 return 給省略

In [89]:
def foo1(value):
  if value:
    return value
  else:
    return None


def foo2(value):
  """Bare return statement implies `return None`"""
  if value:
    return value
  else:
    return

def foo3(value):
  """Missing return statement implies `return None`"""
  if value:
    return value

**Key Takeaways**
+ If a function doesn’t specify a return value, it returns None .
Whether to explicitly return None is a stylistic decision.
+ This is a core Python feature but your code might communicate
its intent more clearly with an explicit return None statement.